# 🧠 Handwritten Digit Recognition using CNN
This project trains a Convolutional Neural Network (CNN) to recognize handwritten digits (0–9) using the MNIST dataset.

The notebook is organized into sections:
1. Data Loading and Preprocessing
2. Model Building and Training
3. Model Evaluation (Accuracy, Precision, Recall, F1-score)
4. Predict Digit from Image Path
5. Predict Digit by Writing on Screen (Real-time Capture)

## 1️⃣ Data Loading and Preprocessing
We load the MNIST dataset and apply normalization, reshaping, and basic data augmentation to improve model performance.

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize
X_train = X_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)


## 2️⃣ Build and Train the CNN Model

In [ ]:

from tensorflow.keras import layers, models

# Define CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(datagen.flow(X_train, y_train_cat, batch_size=64), epochs=5, validation_data=(X_test, y_test_cat))

# Save the model
model.save("digit_model.h5")
print("Model training complete and saved as 'digit_model.h5'")


## 3️⃣ Model Evaluation
We evaluate the model using accuracy, precision, recall, F1 score, and confusion matrix.

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predict and evaluate
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


## 4️⃣ Predict Digit from Image Path
This function loads an image from the given path, preprocesses it, and makes a prediction.

In [ ]:

import cv2
import os

# Preprocessing function
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (28, 28))
    img = cv2.bitwise_not(img)
    img = img.astype('float32') / 255.0
    img = img.reshape(1, 28, 28, 1)
    return img

# Predict from terminal input
def predict_from_path():
    model = tf.keras.models.load_model("digit_model.h5")
    path = input("Enter image path: ").strip()
    if not os.path.exists(path):
        print("❌ File not found!")
        return
    img = preprocess_image(path)
    pred = model.predict(img)
    print(f"✅ Predicted digit: {np.argmax(pred)}")

# Uncomment to run
# predict_from_path()


## 5️⃣ Predict Digit by Writing on Screen (Real-time Capture)
This function captures the screen after you write a digit in an application like Paint. It gives you 10 seconds to write the digit.

In [ ]:

import pyautogui
import time
import numpy as np

# Function to capture the screen after delay and predict
def capture_and_predict():
    print("You have 10 seconds to write the digit in an app like Paint!")
    time.sleep(10)  # Wait for 10 seconds
    
    # Capture the screen area where the drawing occurs
    screenshot = pyautogui.screenshot(region=(0, 0, 300, 300))
    screenshot.save("screen_capture.png")
    
    # Preprocess the screenshot for prediction
    img = preprocess_image("screen_capture.png")
    pred = model.predict(img)
    print(f"✅ Predicted digit: {np.argmax(pred)}")

# Uncomment to run
# capture_and_predict()
